<a href="https://colab.research.google.com/github/hasimoka/BlazeFace-PyTorch/blob/master/InferenceONNXModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/BlazeFace-PyTorch/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/BlazeFace-PyTorch


In [2]:
!pip install onnxruntime

In [10]:
import blazeface_with_onnx as blazeface

model = blazeface.BlazeFaceWithOnnx()
model.load_anchors("./anchors.npy")

In [11]:
import cv2

img = cv2.imread("./1face.png")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

front_detections = model.predict_on_image(img)
print(front_detections)
print(front_detections.shape)

box_a.shape: (1, 4)
box_b.shape: (2, 4)
((box_a[:, 2]-box_a[:, 0])*(box_a[:, 3]-box_a[:, 1]))[:, np.newaxis]: [[0.02780628 0.02780628]]
[[0.2714351  0.317134   0.4415536  0.4872523  0.3863073  0.31266788
  0.46129563 0.3186052  0.43995076 0.35565478 0.432756   0.3914948
  0.3153791  0.32960063 0.47782823 0.3367067  0.99599206]]
(1, 17)
